## Build the Data object

In [2]:
import numpy as np
import pandas as pd 
import random as rd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch_geometric.data as data
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.utils import negative_sampling,train_test_split_edges,to_dense_adj
from sklearn.metrics import roc_auc_score
from torch_geometric.transforms import RandomLinkSplit
from sklearn import preprocessing
from torch_geometric.nn import Node2Vec

device = "cpu"

In [2]:
##Clean and normalize data
###Cole para adestrar
nodes_h1 = pd.read_csv(r"Nodes_t1.csv",sep=",",encoding = 'unicode_escape')
edges_h1 = pd.read_csv(r"Edges_t1.csv",sep=";",encoding = 'unicode_escape')
nodes_h1.drop("ID",axis=1,inplace=True)
##cole para testear
nodes_h2 = pd.read_csv(r"Nodes_t4.csv",sep=",",encoding = 'unicode_escape')
edges_h2 = pd.read_csv(r"Edges_t4.csv",sep=",",encoding = 'unicode_escape')
nodes_h2 = nodes_h2[nodes_h1.columns]
nodes_h1["Escuela"] = "h1"
nodes_h2["Escuela"] = "h2"
edges_h1["Escuela"] = "h1"
edges_h2["Escuela"] = "h2"
#edges_h2.drop("relacion",axis=1,inplace = True)
edges_h2.columns = edges_h1.columns
edges_h1[["from","to"]] = edges_h1[["from","to"]].apply(lambda x:x-edges_h1["from"].min())
edges_h2[["from","to"]] = edges_h2[["from","to"]].apply(lambda x:x+1+edges_h1["from"].max()-edges_h2["from"].min())

In [21]:
datasets_edges[1]["from"].max(),datasets_edges[2]["from"].min()

(645, 646)

In [23]:
len_datasets = 6
datasets_nodes = [0]*len_datasets
datasets_edges = [0]*len_datasets
for i in range(0,len_datasets):
    datasets_nodes[i] = pd.read_csv(r"Nodes_t"+str(i+1)+".csv",sep=",",encoding = 'unicode_escape')
    datasets_edges[i] = pd.read_csv(r"Edges_t"+str(i+1)+".csv",sep=",",encoding = 'unicode_escape')
    datasets_nodes[i].drop("ID",axis=1,inplace=True)
    if i == 0:
        datasets_edges[i][["from","to"]] = datasets_edges[i][["from","to"]].apply(lambda x:x-datasets_edges[i]["from"].min())
    else:
        datasets_edges[i][["from","to"]] = datasets_edges[i][["from","to"]].apply(lambda x:x+1+datasets_edges[i-1]["from"].max()-datasets_edges[i]["from"].min())
    
edges_total = pd.concat(datasets_edges,axis=0).reset_index().drop("index",axis=1)
edges_total = edges_total[edges_total["weight"] != 1].reset_index().drop("index",axis=1)
edges_total["weight"] = edges_total["weight"].apply(lambda x: np.sign(x)).replace({-1:0})
nodes_total = pd.concat(datasets_nodes,axis=0).reset_index().drop("index",axis=1)

In [3]:
edges_total = pd.concat([edges_h1,edges_h2],axis=0).reset_index().drop("index",axis=1)
edges_total = edges_total[edges_total["weight"] != 1].reset_index().drop("index",axis=1)
edges_total["weight"] = edges_total["weight"].apply(lambda x: np.sign(x)).replace({-1:0})
nodes_total = pd.concat([nodes_h1,nodes_h2],axis=0).reset_index().drop("index",axis=1)


### Build the data object

In [4]:
nodes_dummy = pd.get_dummies(nodes_total,drop_first=True)
x = nodes_dummy.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
nodes_norm = pd.DataFrame(x_scaled,columns = nodes_dummy.columns)


In [5]:
total_data = data.Data(x=torch.tensor(nodes_norm.to_numpy(),dtype=torch.float32),
                          edge_index=torch.tensor(edges_total[["from","to"]].to_numpy().T),
                          edge_attr=torch.tensor((pd.get_dummies(edges_total[["weight"]]).to_numpy())))
total_data

Data(x=[2754, 19], edge_index=[2, 28307], edge_attr=[28307, 1])

In [6]:
node_list = list(range(len(nodes_total)))
train_val_set = node_list[0:len(nodes_h1)]
rd.shuffle(train_val_set)
train_set = train_val_set[0:round(len(train_val_set)*0.8)]
val_set = train_val_set[len(train_set):len(train_val_set)]
test_set = node_list[len(nodes_h1):len(nodes_h1)+len(nodes_h2)]
rd.shuffle(test_set)
print(len(train_set),len(test_set),len(val_set))
print(len(train_set)+len(test_set)+len(val_set) == len(nodes_total))

print("train set\t",train_set[:10])
print("test set \t",test_set[:10])
print("val set  \t",val_set[:10])

327 2345 82
True
train set	 [88, 138, 368, 216, 309, 251, 37, 370, 16, 140]
test set 	 [1565, 440, 2578, 1551, 1690, 2634, 1842, 1093, 2456, 2500]
val set  	 [45, 233, 265, 82, 134, 25, 123, 371, 167, 391]


In [7]:
# build test train val masks

train_mask = torch.zeros(len(nodes_total),dtype=torch.long, device=device)
for i in train_set:
    train_mask[i] = 1.

test_mask = torch.zeros(len(nodes_total),dtype=torch.long, device=device)
for i in test_set:
    test_mask[i] = 1.
    
val_mask = torch.zeros(len(nodes_total),dtype=torch.long, device=device)
for i in val_set:
    val_mask[i] = 1.
    
print("train mask \t",train_mask[0:15])
print("test mask  \t",test_mask[0:15])
print("val mask   \t",val_mask[0:15]) 

train mask 	 tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])
test mask  	 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
val mask   	 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])


In [8]:
total_data

Data(x=[2754, 19], edge_index=[2, 28307], edge_attr=[28307, 1])

In [9]:
print("befor\t\t",total_data)
total_data.x = None
total_data.edge_attr = None


befor		 Data(x=[2754, 19], edge_index=[2, 28307], edge_attr=[28307, 1])


In [10]:
# add masks
total_data.train_mask = train_mask
total_data.test_mask = test_mask
total_data.val_mask = val_mask

print("after\t\t",total_data)

after		 Data(edge_index=[2, 28307], train_mask=[2754], test_mask=[2754], val_mask=[2754])


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Node2Vec(total_data.edge_index, embedding_dim=128, walk_length=40,
             context_size=20, walks_per_node=5,
             num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


In [12]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[total_data.train_mask], total_data.y[total_data.train_mask],
                     z[total_data.test_mask], total_data.y[total_data.test_mask],
                     max_iter=10)
    return acc


for epoch in range(1, 121):
    loss = train()
    #acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
        

Epoch: 10, Loss: 0.8907
Epoch: 20, Loss: 0.7447
Epoch: 30, Loss: 0.7285
Epoch: 40, Loss: 0.7244


KeyboardInterrupt: 

In [ ]:
total_data = data.Data(x=torch.tensor(nodes_norm.to_numpy(),dtype=torch.float32),
                          edge_index=torch.tensor(edges_total[["from","to"]].to_numpy().T),
                          edge_attr=torch.tensor((pd.get_dummies(edges_total["weight"]).to_numpy())))
total_data

In [ ]:
z = model()
# from tensor to numpy
emb_128 = z.detach().cpu().numpy()

# convert edge attributes from categorical to numerical
edge_attr_cat = total_data.edge_attr.numpy()
print("Categorical edge attributes:\n",edge_attr_cat[:3])

edge_attr = []
for i in edge_attr_cat:
    edge_attr.append(np.nonzero(i)[0][0])

print("\n\nNumerical edge attributes:\n",edge_attr[:3])

## Compute edge embedding

In [ ]:
edge_embedding = []
for u,v in total_data.edge_index.t():
    edge_embedding.append(np.maximum(emb_128[u],emb_128[v]))

### Make dataframe from embedding and scale

In [ ]:
len_h1 = len(edges_total[edges_total["Escuela"] == "h1"])
emb_df_h1 = pd.concat([pd.DataFrame(edge_embedding[:len_h1]),pd.DataFrame(edge_attr[:len_h1],columns=["label"])],axis=1)

emb_df_h2 = pd.concat([pd.DataFrame(edge_embedding[len_h1:]),pd.DataFrame(edge_attr[len_h1:],columns=["label"])],axis=1)

In [ ]:
emb_df_h1 = emb_df_h1.sample(frac=1)
emb_df_h2 = emb_df_h2.sample(frac=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
sc = MinMaxScaler()

In [ ]:
X_train = emb_df_h1.drop("label",axis=1)
y_train = emb_df_h1["label"]
X_test = emb_df_h2.drop("label",axis=1)
y_test = emb_df_h2["label"]

In [ ]:
sc.fit(X_train)
emb_x_train = sc.transform(X_train)
emb_y_train = y_train
emb_x_test = sc.transform(X_test)
emb_y_test = y_test

### Oversample minority class

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
ros = SMOTE(random_state=0,sampling_strategy="minority")
emb_x_resampled, emb_y_resampled = ros.fit_resample(emb_x_train, emb_y_train)

### Training and prediction 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(max_depth=7,random_state=10,class_weight="balanced")

In [ ]:
clf.fit(emb_x_resampled,emb_y_resampled)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
fig= plt.figure()
ax = fig.add_axes([0,0,1,1])
sns.heatmap(np.round(confusion_matrix(emb_y_test,np.around(clf.predict(emb_x_test)))/len(emb_y_test)*100,2),
            annot=True,ax=ax,cmap="coolwarm")
ax.set_xlabel("Predicted")
ax.set_ylabel("Current")
print("\n")
print(classification_report(emb_y_test,clf.predict(emb_x_test)))


In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128,activation="relu",input_shape=(emb_x_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=10e-5),
         loss="binary_crossentropy",
             metrics=["accuracy"])
model_history = model.fit(emb_x_resampled,emb_y_resampled,validation_data=(emb_x_test,emb_y_test),epochs=250,verbose=1,batch_size=128,
                         #callbacks=[tf.keras.callbacks.EarlyStopping(patience=50)])
                         )

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
fig= plt.figure()
ax = fig.add_axes([0,0,1,1])
sns.heatmap(np.round(confusion_matrix(emb_y_test,np.around(model.predict(emb_x_test)))/len(emb_y_test)*100,2),
            annot=True,ax=ax,cmap="coolwarm")
ax.set_xlabel("Predicted")
ax.set_ylabel("Current")
print("\n")
print(classification_report(emb_y_test,np.around(model.predict(emb_x_test))))


In [ ]:
fig = plt.figure(figsize=(12,8))
plt.rcParams.update({'font.size': 15})
pd.DataFrame(model_history.history)[["loss","val_loss"]].plot(figsize=(10,6))

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.rcParams.update({'font.size': 15})
pd.DataFrame(model_history.history)[["accuracy","val_accuracy"]].plot(figsize=(10,6))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# fit and transform using PCA
pca = PCA(n_components=2)
edge_emb2d = pca.fit_transform(edge_embedding)



df = pd.DataFrame(dict(edge_att=edge_attr))
colors = {0:"red",1:"blue"}
plt.title("edge embedding in 2D")
plt.scatter(edge_emb2d[:,0],edge_emb2d[:,1],c=df.edge_att.map(colors),alpha=0.2)
plt.show()

# not so good but we are using PCA to reduce the dim from 128 to 2

In [26]:
emb_128

NameError: name 'emb_128' is not defined